#  Probar con 2017 y 2016

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [3]:
pruebas = pd.read_csv('properati_dataset_modificado.csv')

In [7]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000
precio_max = 3000000

propiedades = pruebas
p = propiedades.loc[  (propiedades.superficie >= sup_max) | (propiedades.superficie <= sup_min) |\
                      (propiedades.lat >= lat_max) | (propiedades.lat <= lat_min) |\
                       (propiedades.lon >= lon_max) | (propiedades.lon <= lon_min),:]

columnas = ['superficie','lat', 'lon']

## Hay 2000 propiedades con datos "malos" en las pruebas a analizar

## Quizas por eso tenemos muy malos resultados

In [8]:
sup = propiedades.loc[  (propiedades.superficie >= sup_max) | (propiedades.superficie <= sup_min),:]
lat = propiedades.loc[  (propiedades.lat >= lat_max) | (propiedades.lat <= lat_min) ,:]
lon = propiedades.loc[  (propiedades.lon >= lon_max) | (propiedades.lon <= lon_min),:]

columnas = ['superficie','lat', 'lon']

In [9]:
s = sup.loc[:,'id'].count()
l = lat.loc[:,'id'].count()
lo = lon.loc[:,'id'].count()

In [10]:
print("Mal superficie   {}\nMal Latitud   {}\nMal longitud   {}".format(s,l,lo))

Mal superficie   532
Mal Latitud   70
Mal longitud   637


### El principal problema es la superficie

In [9]:
def normalizar_sup(s):
    if s < sup_min:
        return sup_min
    if s > sup_max:
        return sup_max
    return s

def normalizar_lat(s):
    if s > lat_min and s < lat_max:
        return s
    return propiedades.loc[:,'lat'].mean()

def normalizar_lon(s):
    if s > lon_min and s < lon_max:
        return s
    return propiedades.loc[:,'lon'].mean()

sup_min = 10
sup_max = 350
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000
precio_max = 3000000

In [10]:
pruebas.loc[:,'sup350'] = pruebas.loc[:,'superficie'].apply(lambda x: normalizar_sup(x))
pruebas.loc[:,'sup350'].describe()

count    14166.000000
mean       114.052525
std         94.170425
min         10.000000
25%         47.000000
50%         74.000000
75%        150.000000
max        350.000000
Name: sup350, dtype: float64

In [17]:
pruebas.loc[:,'lat'] = pruebas.loc[:,'lat'].apply(lambda x: normalizar_lat(x))
pruebas.loc[:,'lon'] = pruebas.loc[:,'lon'].apply(lambda x: normalizar_lon(x))
pruebas.loc[:,['lat','lon','superficie']].describe()

,lat,lon,superficie
count,14166.000000,14166.000000,14166.000000
mean,-34.633563,-58.481002,110.423625
std,0.090780,0.124581,85.621742
min,-34.991982,-58.975341,10.000000
25%,-34.651573,-58.510157,47.000000
50%,-34.612545,-58.452145,74.000000
75%,-34.587729,-58.412486,150.000000
max,-34.112857,-58.008589,300.000000


In [12]:
pruebas.to_csv('properati_dataset_modificado2.csv', index = False)

In [11]:
#p = pd.read_csv('properati_dataset_modificado2.csv')
#p.loc[:,'sup250'] = pruebas.loc[:,'sup250']
#p.loc[:,'sup350'] = pruebas.loc[:,'sup350']
#p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 35 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
state_name                 14166 non-null object
lat                        14166 non-null float64
lon                        14166 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
seguridad                  14166 non-null bool
gimnasio                   14166 non-null bool
aire                       14166 non-null bool
pileta                     14166 non-null bool
cochera        

# Hago algunas pruebas

In [2]:
propiedades = pd.read_csv('../set_datos_propiedades.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.7)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 75, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('prueba_mas_menos_datos_datasetmodificado2.csv', index = False)

In [5]:
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.272705e+05
std,4.290968e+04,2.110054e+05
min,3.632000e+03,5.000000e+04
25%,2.271809e+06,1.090349e+05
50%,2.290284e+06,1.622950e+05
75%,2.309395e+06,2.663533e+05
max,2.332338e+06,2.290000e+06


# --------------------------------------------------

# ---------------------------------------------------

In [6]:
propiedades = pd.read_csv('../datos_con_fecha.csv')
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                #         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

#propiedades.loc[:,'fecha'] = propiedades.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

columnas = ['lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera', 'fecha'\
           ,'transporte','comercio', 'servicios', 'doble piso', 'jardin']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [7]:
# Parametros

criterios = ['friedman_mse']
max_depth = [75]
features = [1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 3
arbol = DecisionTreeRegressor()

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_


print("Precision: {:.2f} \n Parametros = {}".format(score,mejores_parametros))

Precision: 88.51 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': 75, 'max_features': 1.0}


In [8]:
analizar = pd.read_csv("properati_dataset_modificado2.csv")

analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('prueba_decision_tree_sin_superficie_mas_menos_datos.csv', index = False)

# ---------------------------------------------------

In [3]:
propiedades = pd.read_csv('../datos_con_fecha.csv')
sup_min = 10
sup_max = 200
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000  #
precio_max = 500000 #

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                #         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

#propiedades.loc[:,'fecha'] = propiedades.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

columnas = ['lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera', 'fecha'\
           ,'transporte','comercio', 'servicios', 'doble piso', 'jardin']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [4]:
# Parametros

criterios = ['friedman_mse']
max_depth = [75]
features = [1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 3
arbol = DecisionTreeRegressor()

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_


print("Precision: {:.2f} \n Parametros = {}".format(score,mejores_parametros))

Precision: 88.03 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': 75, 'max_features': 1.0}


In [5]:
analizar = pd.read_csv("properati_dataset_modificado2.csv")

analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('prueba_decision_tree_sin_superficie_menos_menos_datos.csv', index = False)

In [6]:
X = propiedades.loc[:,columnas]
y = propiedades.loc[:,'price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

precision = grid.score(X_test,y_test) * 100
error = mean_squared_error(grid.predict(X_test),y_test)
print("Precision: {:.2f}%  ,   Error = {:.0f},   dig = {}".format(precision,error,len(str(int(round(error,0))))))

Precision: 96.32%  ,   Error = 432160718,   dig = 9


# ---------------------------------------------------

In [19]:
propiedades = pd.read_csv('../datos_con_fecha.csv')
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 20000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera'\
           ,'transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
# Parametros
criterios = ['friedman_mse']
max_depth = [75]
features = [1.0]
parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}
iteraciones_cross_validation = 3
arbol = DecisionTreeRegressor()
grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)
grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)
score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
print("Precision: {:.2f} \n Parametros = {}".format(score,mejores_parametros))

Precision: 96.21 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': 75, 'max_features': 1.0}


In [ ]:
analizar = pd.read_csv("properati_dataset_modificado.csv")

analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('prueba_aumento_preciomin_supmax_datasetModificado1_sinfecha.csv', index = False)

In [20]:
analizar = pd.read_csv("properati_dataset_modificado2.csv")
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera'\
           ,'transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']
analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('prueba_preciomin_20k.csv', index = False)

In [ ]:
#MEJORES RESTRICCIONES HASTA AHORA

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 50000
precio_max = 3000000